# Humidity Controll

In [1]:
%reset -f

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import psychro as psy
import pandas as pd
import va_hum

# Description:
In this project, the goal is to implement a humidity control for a room. The aim is to ensure that both parameters temperature and humidity remain stable over time, regardless of changing outdoor conditions in winter or summer. Such a humidity control is needed in environments where sensitive items are stored, for example in a museum (to protect artworks), archives, laboratories, or in data centers where precise climate control is critical.

The system includes a mixing box for heat recovery, an air cooler for cooling and dehumidification, a vapor humidifier and an air heater. Additionally, it comprises the thermal zone, the building, and the two controllers for indoor temperature and for indoor humidity.


![AHU](cool_CAV.svg)

# Humidity control logic:
The relative humidity of the outdoor air is measured, from which the absolute humidity is calculated (`w_0`).

If the absolute humidity of the outdoor air (`w_0`) is lower than the desired indoor setpoint (`w_6_sp`), the air is humidified by the vapor humidifier and then heated by the air heater to achieve the needed supply air temperature.

If the absolute humidity of the outdoor air (`w_0`) is higher than the indoor setpoint (`w_6_sp`), the air is dehumidified by the air cooler and subsequently heated by the air heater to reach the needed supply air temperature.


if $w_{0}>w_{6,sp}$ → dehuminification → MX1, CC, MX2, HC, TZ, BL

if $\omega_{0}<w_{6,sp},$ → huminification → MX1, HC

## if dehuminification

### Mixing MX1
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{1}}=\alpha \cdot \dot{m} \cdot c \cdot \theta_{0} + (1-\alpha) \cdot \dot{m} \cdot c \cdot \textcolor{red}{\theta_{6}}$  
$\dot{m} \cdot l\cdot \textcolor{red}{w_{1}} = \alpha \cdot \dot{m} \cdot l \cdot w_0+(1-\alpha)\cdot \dot{m} \cdot l \cdot \textcolor{red}{w_{6}}$

### Cooling with dehumidification CC
$\alpha \cdot \dot{m} \cdot c \cdot \textcolor{red}{\theta_{2}}=\alpha \cdot \dot{m} \cdot\textcolor{red}{\theta_{1}} + \textcolor{red}{Q_{s,CC}}$  
$\alpha \cdot \dot{m} \cdot l \cdot \textcolor{red}{w_{2}} = \alpha \cdot \dot{m} \cdot \textcolor{red}{w_{1}} + \textcolor{red}{Q_{l,CC}}$  
$\textcolor{red}{w_2} = f(\theta_{s})= f'_{\theta_{2}^{0}} \cdot \textcolor{red}{\theta_{2}}-\textcolor{red}{w_{2}} = f'_{\theta_{2}^{0}}\cdot\theta_{2,0}-w_{2,0}$  
$\textcolor{red}{Q_{t,CC}}=\textcolor{red}{Q_{s,CC}}+\textcolor{red}{Q_{l,CC}}$

### Mixing after dehumidification MX2
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{3}}=\beta \cdot \dot{m} \cdot \textcolor{red}{\theta_{2}} + (1-\beta) \cdot \dot{m} \cdot \textcolor{red}{\theta_{1}}$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{3}}=\beta \cdot \dot{m} \cdot \textcolor{red}{w_{2}} + (1-\beta) \cdot \dot{m} \cdot \textcolor{red}{w_{1}}$

### Heating coil HC
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{4}} = \dot{m} \cdot c \cdot \textcolor{red}{\theta_{3}} + \textcolor{red}{Q_{s,HC}}$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{4}} = \dot{m} \cdot l \cdot \textcolor{red}{w_{3}}$

### vapor humidification
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{5}} = \dot{m} \cdot c \cdot \textcolor{red}{\theta_{4}}$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{5}} = \dot{m} \cdot l \cdot \textcolor{red}{w_{4}}$

### Thermal Zone
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{6}} = \dot{m} \cdot c \cdot \textcolor{red}{\theta_{5}} + \textcolor{red}{Q_{s,B}}$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{6}} = \dot{m} \cdot c \cdot \textcolor{red}{w_{5}} + \textcolor{red}{Q_{l,B}}$

### Building
$\textcolor{red}{Q_{s,B}} =  (UA + \dot{m}_i \cdot c) \cdot ( \theta_0 - \textcolor{red}{\theta_6}) + Q_{s,a}$  
$\textcolor{red}{Q_{l,B}} = \dot{m}_i \cdot l \cdot (w_0-\textcolor{red}{w_6}) + Q_{l,a}$

### Indoor temperture controller
$K\theta \cdot \theta_{6,sp} = K\theta \cdot \textcolor{red}{\theta_{6}} + \textcolor{red}{Q_{s,HC}}$

### Indoor humidity controller
$Kw \cdot w_{6,sp} = Kw \cdot \textcolor{red}{w_{6}} + \textcolor{red}{Q_{l,CC}}$

x = Vector of 18 unknowns:  
$\textcolor{red}{\theta_{1}},\textcolor{red}{w_{1}},\textcolor{red}{\theta_{2}},\textcolor{red}{w_{2}},\textcolor{red}{\theta_{3}},\textcolor{red}{w_{3}},\textcolor{red}{\theta_{4}},\textcolor{red}{w_{4}},\textcolor{red}{\theta_{5}},\textcolor{red}{w_{5}},\textcolor{red}{\theta_{6}},\textcolor{red}{w_{6}},$  
$\textcolor{red}{Q_{s,CC}}, \textcolor{red}{Q_{l,CC}}, \textcolor{red}{Q_{t_CC}}, \textcolor{red}{Q_{s,HC}}, \textcolor{red}{Q_{s,B}}, \textcolor{red}{Q_{l,B}}$


## if humification

### Mixing MX1
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{1}}=\alpha \cdot \dot{m} \cdot c \cdot \theta_{0} + (1-\alpha) \cdot \dot{m} \cdot c \cdot \textcolor{red}{\theta_{6}}$  
$\dot{m} \cdot l\cdot \textcolor{red}{w_{1}} = \alpha \cdot \dot{m} \cdot l \cdot w_0+(1-\alpha)\cdot \dot{m} \cdot l \cdot \textcolor{red}{w_{6}}$

### Cooling with dehumidification CC
$\alpha \cdot \dot{m} \cdot c \cdot \textcolor{red}{\theta_{2}}=\alpha \cdot \dot{m} \cdot\textcolor{red}{\theta_{1}}$  
$\alpha \cdot \dot{m} \cdot l \cdot \textcolor{red}{w_{2}} = \alpha \cdot \dot{m} \cdot \textcolor{red}{w_{1}}$  

### Mixing after dehumidification MX2
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{3}}=\beta \cdot \dot{m} \cdot \textcolor{red}{\theta_{2}} + (1-\beta) \cdot \dot{m} \cdot \textcolor{red}{\theta_{1}}$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{3}}=\beta \cdot \dot{m} \cdot \textcolor{red}{w_{2}} + (1-\beta) \cdot \dot{m} \cdot \textcolor{red}{w_{1}}$

### Heating coil HC
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{4}} = \dot{m} \cdot c \cdot \textcolor{red}{\theta_{3}} + \textcolor{red}{Q_{s,HC}}$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{4}} = \dot{m} \cdot l \cdot \textcolor{red}{w_{3}}$

### vapor himidification
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{5}} = \dot{m} \cdot c \cdot \textcolor{red}{\theta_{4}}$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{5}} = \dot{m} \cdot l \cdot \textcolor{red}{w_{4}} + \textcolor{red}{Q_{l,VH}}$

### Thermal Zone
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{6}} = \dot{m} \cdot c \cdot \textcolor{red}{\theta_{5}} + \textcolor{red}{Q_{s,B}}$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{6}} = \dot{m} \cdot c \cdot \textcolor{red}{w_{5}} + \textcolor{red}{Q_{l,B}}$

### Building
$\textcolor{red}{Q_{s,B}} =  (UA + \dot{m}_i \cdot c) \cdot ( \theta_0 - \textcolor{red}{\theta_6}) + Q_{s,a}$  
$\textcolor{red}{Q_{l,B}} = \dot{m}_i \cdot l \cdot (w_0-\textcolor{red}{w_6}) + Q_{l,a}$

### Indoor temperture controller
$K\theta \cdot \theta_{6,sp} = K\theta \cdot \textcolor{red}{\theta_{6}} + \textcolor{red}{Q_{s,HC}}$

### Indoor humidity controller
$Kw \cdot w_{6,sp} = Kw \cdot \textcolor{red}{w_{6}} + \textcolor{red}{Q_{l,VH}}$

x = Vector of 16 unknowns:  
$\textcolor{red}{\theta_{1}},\textcolor{red}{w_{1}},\textcolor{red}{\theta_{2}},\textcolor{red}{w_{2}},\textcolor{red}{\theta_{3}},\textcolor{red}{w_{3}},\textcolor{red}{\theta_{4}},\textcolor{red}{w_{4}},\textcolor{red}{\theta_{5}},\textcolor{red}{w_{5}},\textcolor{red}{\theta_{6}},\textcolor{red}{w_{6}},$  
$\textcolor{red}{Q_{s,HC}}, \textcolor{red}{Q_{l,VH}}, \textcolor{red}{Q_{s,B}}, \textcolor{red}{Q_{l,B}}$


In [3]:
# constants
l = 2496e3          # specific latent heat of vaporization [J/kg]
c = 1e3             # J/(kg·K), specific heat of dry air
ρ = 1.2             # kg/m3, density of air

# parameters
ϕ_0 = 0.30           # relative humidity of outside air [-]
θ_0 = 35            # temperature of outside Air [°C]
θ_6_sp = 21         # temperature setpoint of the room [°C]
ϕ_6_sp = 0.55       # relative humidity setpoint of the room [-]

α = 0.5             # mixing ratio of outdoor air [-]
m = 2.8             # mass flow rate of supply dry air [kg/s]
β = 0.1             # mixing ratio dehumidification [-]

UA = 150 # total transmittance [W/K]
m_i = 1 # infiltration mass flow [kg/s]
Q_aux_s = 4000 # auxiliary heat sensible [W]
Q_aux_l = 3000 # auxiliary  latent gain [W]

Kθ, Kw = 1e10, 1e6         # controller gain

θ_s0 = 10       # °C, initial guess for saturation temperature

In [4]:
# humidity ratios
w_6_sp = psy.w(θ_6_sp, ϕ_6_sp)  # humidity ratio setpoint of the room [kg/kg]
w_0 = psy.w(θ_0, ϕ_0)  # humidity ratio of outside air [kg/kg]
print(f"humidity ratio setpoint of the room: w_0 = {w_0:.4f} kg/kg")
print(f"humidity ratio setpoint of the room: w_6_sp = {w_6_sp:.4f} kg/kg")

humidity ratio setpoint of the room: w_0 = 0.0105 kg/kg
humidity ratio setpoint of the room: w_6_sp = 0.0085 kg/kg


In [5]:
 # Define column names (unknowns)
columns_DH = [
    'θ1', 'w1', 'θ2', 'w2', 'θ3', 'w3', 'θ4', 'w4','θ5', 'w5','θ6', 'w6',
    'Qs_CC', 'Ql_CC', 'Qt_CC', 'Qs_HC', 'Qs_B', 'Ql_B'
]

# Define row names (equations) with 's' for sensible and 'l' for latent
rows_DH = [
    'MXs', 'MXl',                    # Mixing box equations
    'CCs', 'CCl', 'DH', 'DH_Q',      # Colling with dehumidification
    'MX2s', 'MX2l',                  # Mixing after dehumidification
    'HCs', 'HCl',                    # Heating coil equations
    'VHs', 'VHl',                    # Vapor humidifier equations

    'TZs', 'TZl',                   # Thermal zone equations
    'BLs', 'BLl',                   # Building equations
    'Kθ', 'Kw'                      # Controller equations
]

# Initialize coefficient matrix A and vector b
A_DH = pd.DataFrame(0.0, index=rows_DH, columns=columns_DH)
b_DH = pd.Series(0.0, index=rows_DH)


## if dehuminification

### Mixing MX1
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{1}} - (1-\alpha) \cdot \dot{m} \cdot c \cdot \textcolor{red}{\theta_{6}} = \alpha \cdot \dot{m} \cdot c \cdot \theta_{0}$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{1}} - (1-\alpha) \cdot \dot{m} \cdot l \cdot \textcolor{red}{w_{6}} = \alpha \cdot \dot{m} \cdot l \cdot w_0$

### Cooling with dehumidification CC
$\alpha \cdot \dot{m} \cdot c \cdot \textcolor{red}{\theta_{2}} - \alpha \cdot \dot{m} \cdot c \cdot \textcolor{red}{\theta_{1}} - \textcolor{red}{Q_{s,CC}} = 0$  
$\alpha \cdot \dot{m} \cdot l \cdot \textcolor{red}{w_{2}} - \alpha \cdot \dot{m} \cdot l \cdot \textcolor{red}{w_{1}} - \textcolor{red}{Q_{l,CC}} = 0$  
$f'_{\theta_{2}^{0}} \cdot \textcolor{red}{\theta_{2}} - \textcolor{red}{w_{2}} = f'_{\theta_{2}^{0}}\cdot\theta_{2,0} - w_{2,0}$  
$\textcolor{red}{Q_{t,CC}} - \textcolor{red}{Q_{s,CC}} - \textcolor{red}{Q_{l,CC}} = 0$

### Mixing after dehumidification MX2
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{3}} - \beta \cdot \dot{m} \cdot c \cdot \textcolor{red}{\theta_{2}} - (1-\beta) \cdot \dot{m} \cdot c \cdot \textcolor{red}{\theta_{1}} = 0$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{3}} - \beta \cdot \dot{m} \cdot l \cdot \textcolor{red}{w_{2}} - (1-\beta) \cdot \dot{m} \cdot l \cdot \textcolor{red}{w_{1}} = 0$  

### Heating coil HC
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{4}} - \dot{m} \cdot c \cdot \textcolor{red}{\theta_{3}} - \textcolor{red}{Q_{s,HC}} = 0$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{4}} - \dot{m} \cdot l \cdot \textcolor{red}{w_{3}} = 0$

### Vapor humidification
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{5}} - \dot{m} \cdot c \cdot \textcolor{red}{\theta_{4}} = 0$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{5}} - \dot{m} \cdot l \cdot \textcolor{red}{w_{4}} = 0$

### Thermal Zone
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{6}} - \dot{m} \cdot c \cdot \textcolor{red}{\theta_{5}} - \textcolor{red}{Q_{s,B}} = 0$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{6}} - \dot{m} \cdot l \cdot \textcolor{red}{w_{5}} - \textcolor{red}{Q_{l,B}} = 0$

### Building
$\textcolor{red}{Q_{s,B}} + (UA + \dot{m}_i \cdot c) \cdot \textcolor{red}{\theta_6} = (UA + \dot{m}_i \cdot c) \cdot \theta_0 + Q_{s,a}$  
$\textcolor{red}{Q_{l,B}} + \dot{m}_i \cdot l \cdot \textcolor{red}{w_6} = \dot{m}_i \cdot l \cdot w_0 + Q_{l,a}$

### Indoor temperture controller
$K\theta \cdot \textcolor{red}{\theta_{6}} + \textcolor{red}{Q_{s,HC}} = K\theta \cdot \theta_{6,sp}$

### Indoor humidity controller
$Kw \cdot \textcolor{red}{w_{6}} + \textcolor{red}{Q_{l,CC}} = Kw \cdot w_{6,sp}$



In [6]:
# --------------------
# Mixing box equations
# --------------------

# sensible
A_DH.loc['MXs', 'θ1'] = m * c
A_DH.loc['MXs', 'θ6'] = -(1 - α) * m * c
b_DH.loc['MXs'] = α * m * c * θ_0

# latent
A_DH.loc['MXl', 'w1'] = m * l
A_DH.loc['MXl', 'w6'] = -(1 - α) * m * l
b_DH.loc['MXl'] = α * m * l * w_0

# ---------------------------------------
# Cooling with dehumidification equations
# ----------------------------------------

# sensible
A_DH.loc['CCs', 'θ2'] =  m * c
A_DH.loc['CCs', 'θ1'] =  -m * c
A_DH.loc['CCs', 'Qs_CC'] = -1
b_DH.loc['CCs'] = 0

# latent
A_DH.loc['CCl', 'w2'] =  m * l
A_DH.loc['CCl', 'w1'] =  -m * l
A_DH.loc['CCl', 'Ql_CC'] = -1
b_DH.loc['CCl'] = 0

# load equation
A_DH.loc['DH_Q', 'Qt_CC'] = 1
A_DH.loc['DH_Q', 'Qs_CC'] = -1
A_DH.loc['DH_Q', 'Ql_CC'] = -1
b_DH.loc['DH_Q'] = 0

# ----------------------------------------
# Mixing after dehumidification equations
# ----------------------------------------

# sensible
A_DH.loc['MX2s', 'θ3'] = m * c
A_DH.loc['MX2s', 'θ2'] = -β * m *c
A_DH.loc['MX2s', 'θ1'] = -(1 - β) * m * c
b_DH.loc['MX2s'] = 0

# latent
A_DH.loc['MX2l', 'w3'] = m * l
A_DH.loc['MX2l', 'w2'] = -β * m * l
A_DH.loc['MX2l', 'w1'] = -(1 - β) * m * l
b_DH.loc['MX2l'] = 0

# ----------------------
# Heating coil equations
# ----------------------

# sensible
A_DH.loc['HCs', 'θ4'] = m * c
A_DH.loc['HCs', 'θ3'] = -m * c
A_DH.loc['HCs', 'Qs_HC'] = -1
b_DH.loc['HCs'] = 0

# latent
A_DH.loc['HCl', 'w4'] = m * l
A_DH.loc['HCl', 'w3'] = -m * l
b_DH.loc['HCl'] = 0

# --------------------------
# Vapor humidifier equations
# --------------------------

# sensible
A_DH.loc['VHs', 'θ5'] = m * c
A_DH.loc['VHs', 'θ4'] = -m * c
b_DH.loc['VHs'] = 0

# latent
A_DH.loc['VHl', 'w5'] = m * l
A_DH.loc['VHl', 'w4'] = -m * l
b_DH.loc['VHl'] = 0

# ------------------------
# Thermal zone equations
# ------------------------

# sensible
A_DH.loc['TZs', 'θ6'] = m * c
A_DH.loc['TZs', 'θ5'] = -m * c
A_DH.loc['TZs', 'Qs_B'] = -1
b_DH.loc['TZs'] = 0

# latent
A_DH.loc['TZl', 'w6'] = m * l
A_DH.loc['TZl', 'w5'] = -m * l
A_DH.loc['TZl', 'Ql_B'] = -1
b_DH.loc['TZl'] = 0

# ------------------------  
# Building equations
# ------------------------

# sensible
A_DH.loc['BLs', 'Qs_B'] = 1
A_DH.loc['BLs', 'θ6'] = (UA + m_i * c)
b_DH.loc['BLs'] = (UA + m_i * c) * θ_0 + Q_aux_s

# latent
A_DH.loc['BLl', 'Ql_B'] = 1
A_DH.loc['BLl', 'w6'] =(m_i * l)
b_DH.loc['BLl'] = (m_i * l) * w_0 + Q_aux_l

# ------------------------
# Controller equations
# ------------------------

# temperature controller
A_DH.loc['Kθ', 'θ6'] = Kθ
A_DH.loc['Kθ', 'Qs_HC'] = 1
b_DH.loc['Kθ'] = Kθ * θ_6_sp

# humidity controller
A_DH.loc['Kw', 'w6'] = Kw
A_DH.loc['Kw', 'Ql_CC'] = 1
b_DH.loc['Kw'] = Kw * w_6_sp



# Humification

## if humification

### Mixing MX1
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{1}} - (1-\alpha) \cdot \dot{m} \cdot c \cdot \textcolor{red}{\theta_{6}} = \alpha \cdot \dot{m} \cdot c \cdot \theta_{0}$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{1}} - (1-\alpha) \cdot \dot{m} \cdot l \cdot \textcolor{red}{w_{6}} = \alpha \cdot \dot{m} \cdot l \cdot w_0$

### Cooling with dehumidification CC
$\alpha \cdot \dot{m} \cdot c \cdot \textcolor{red}{\theta_{2}} - \alpha \cdot \dot{m} \cdot c \cdot \textcolor{red}{\theta_{1}} = 0$  
$\alpha \cdot \dot{m} \cdot l \cdot \textcolor{red}{w_{2}} - \alpha \cdot \dot{m} \cdot l \cdot\textcolor{red}{w_{1}} = 0$  

### Mixing after dehumidification MX2
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{3}} - \beta \cdot \dot{m} \cdot c \cdot \textcolor{red}{\theta_{2}} - (1-\beta) \cdot \dot{m} \cdot c \cdot \textcolor{red}{\theta_{1}} = 0$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{3}} - \beta \cdot \dot{m} \cdot l \cdot \textcolor{red}{w_{2}} - (1-\beta) \cdot \dot{m} \cdot l \cdot \textcolor{red}{w_{1}} = 0$

### Heating coil HC
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{4}} - \dot{m} \cdot c \cdot \textcolor{red}{\theta_{3}} - \textcolor{red}{Q_{s,HC}} = 0$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{4}} - \dot{m} \cdot l \cdot \textcolor{red}{w_{3}} = 0$

### Vapor humidification
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{5}} - \dot{m} \cdot c \cdot \textcolor{red}{\theta_{4}} = 0$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{5}} - \dot{m} \cdot l \cdot \textcolor{red}{w_{4}} - \textcolor{red}{Q_{l,VH}} = 0$

### Thermal Zone
$\dot{m} \cdot c \cdot \textcolor{red}{\theta_{6}} - \dot{m} \cdot c \cdot \textcolor{red}{\theta_{5}} - \textcolor{red}{Q_{s,B}} = 0$  
$\dot{m} \cdot l \cdot \textcolor{red}{w_{6}} - \dot{m} \cdot l \cdot \textcolor{red}{w_{5}} - \textcolor{red}{Q_{l,B}} = 0$

### Building
$\textcolor{red}{Q_{s,B}} + (UA + \dot{m}_i \cdot c) \cdot \textcolor{red}{\theta_6} = (UA + \dot{m}_i \cdot c) \cdot \theta_0 + Q_{s,a}$  
$\textcolor{red}{Q_{l,B}} + \dot{m}_i \cdot l \cdot \textcolor{red}{w_6} = \dot{m}_i \cdot l \cdot w_0 + Q_{l,a}$

### Indoor temperture controller
$K\theta \cdot \textcolor{red}{\theta_{6}} + \textcolor{red}{Q_{s,HC}} = K\theta \cdot \theta_{6,sp}$

### Indoor humidity controller
$Kw \cdot \textcolor{red}{w_{6}} + \textcolor{red}{Q_{l,VH}} = Kw \cdot w_{6,sp}$

In [7]:
 # Define column names (unknowns)
columns_H = [
    'θ1', 'w1', 'θ2', 'w2', 'θ3', 'w3', 'θ4', 'w4','θ5', 'w5','θ6', 'w6',
    'Qs_HC', 'Ql_VC', 'Qs_B', 'Ql_B'
]

# Define row names (equations) with 's' for sensible and 'l' for latent
rows_H = [
    'MXs', 'MXl',                    # Mixing box equations
    'CCs', 'CCl',                    # Colling with dehumidification
    'MX2s', 'MX2l',                  # Mixing after dehumidification
    'HCs', 'HCl',                    # Heating coil equations
    'VHs', 'VHl',                    # Vapor humidifier equations

    'TZs', 'TZl',                   # Thermal zone equations
    'BLs', 'BLl',                   # Building equations
    'Kθ', 'Kw'                      # Controller equations
]

# Initialize coefficient matrix A and vector b
A_H = pd.DataFrame(0.0, index=rows_H, columns=columns_H)
b_H = pd.Series(0.0, index=rows_H)


In [8]:
# --------------------
# Mixing box equations
# --------------------

# sensible
A_H.loc['MXs', 'θ1'] = m * c
A_H.loc['MXs', 'θ6'] = -(1 - α) * m * c
b_H.loc['MXs'] = α * m * c * θ_0

# latent
A_H.loc['MXl', 'w1'] = m * l
A_H.loc['MXl', 'w6'] = -(1 - α) * m * l
b_H.loc['MXl'] = α * m * l * w_0

# ---------------------------------------
# Cooling with dehumidification equations
# ----------------------------------------

# sensible
A_H.loc['CCs', 'θ2'] =  m * c
A_H.loc['CCs', 'θ1'] = - m * c
b_H.loc['CCs'] = 0

# latent
A_H.loc['CCl', 'w2'] =  m * l
A_H.loc['CCl', 'w1'] = -m * l
b_H.loc['CCl'] = 0

# ----------------------------------------
# Mixing after dehumidification equations
# ----------------------------------------

# sensible
A_H.loc['MX2s', 'θ3'] = m * c
A_H.loc['MX2s', 'θ2'] = -β * m *c
A_H.loc['MX2s', 'θ1'] = -(1 - β) * m * c
b_H.loc['MX2s'] = 0

# latent
A_H.loc['MX2l', 'w3'] = m * l
A_H.loc['MX2l', 'w2'] = -β * m * l
A_H.loc['MX2l', 'w1'] = -(1 - β) * m * l
b_H.loc['MX2l'] = 0

# ----------------------
# Heating coil equations
# ----------------------

# sensible
A_H.loc['HCs', 'θ4'] = m * c
A_H.loc['HCs', 'θ3'] = -m * c
A_H.loc['HCs', 'Qs_HC'] = -1
b_H.loc['HCs'] = 0
# latent
A_H.loc['HCl', 'w4'] = m * l
A_H.loc['HCl', 'w3'] = -m * l
b_H.loc['HCl'] = 0
# --------------------------
# Vapor humidifier equations
# --------------------------

# sensible
A_H.loc['VHs', 'θ5'] = m * c
A_H.loc['VHs', 'θ4'] = -m * c
b_H.loc['VHs'] = 0

# latent
A_H.loc['VHl', 'w5'] = m * l
A_H.loc['VHl', 'w4'] = -m * l
A_H.loc['VHl', 'Ql_VC'] = -1
b_H.loc['VHl'] = 0
# -----------------------
# Thermal zone equations
# ------------------------

# sensible
A_H.loc['TZs', 'θ6'] = m * c
A_H.loc['TZs', 'θ5'] = -m * c
A_H.loc['TZs', 'Qs_B'] = -1
b_H.loc['TZs'] = 0

# latent
A_H.loc['TZl', 'w6'] = m * l
A_H.loc['TZl', 'w5'] = -m * l
A_H.loc['TZl', 'Ql_B'] = -1
b_H.loc['TZl'] = 0

# ------------------------
# Building equations
# ------------------------

# sensible
A_H.loc['BLs', 'Qs_B'] = 1
A_H.loc['BLs', 'θ6'] = (UA + m_i * c)
b_H.loc['BLs'] = (UA + m_i * c) * θ_0 + Q_aux_s

# latent
A_H.loc['BLl', 'Ql_B'] = 1
A_H.loc['BLl', 'w6'] = (m_i * l)
b_H.loc['BLl'] = (m_i * l) * w_0 + Q_aux_l

# ------------------------
# Controller equations
# ------------------------

# temperature controller
A_H.loc['Kθ', 'θ6'] = Kθ
A_H.loc['Kθ', 'Qs_HC'] = 1
b_H.loc['Kθ'] = Kθ * θ_6_sp

# humidity controller
A_H.loc['Kw', 'w6'] = Kw
A_H.loc['Kw', 'Ql_VC'] = 1
b_H.loc['Kw'] = Kw * w_6_sp



# Solving Simulation

In [9]:
if w_0 > w_6_sp:
    print('Dehumidification')
    print('======================')

    theta_s_guess = θ_s0  # Global initial guess for saturation temperature

    delta_ws = 0.001  # Initial difference to start iterations
    tolerance_w = 0.01e-10 # Tolerance for convergence in humidity ratio [kg/kg]
    max_iterations_sat = 100  # Maximum number of iterations to prevent infinite loops
    iteration_count = 0  # Initialize iteration counter
    
    converged_sat = False # Flag to check if saturation iteration converged
    solution_series_DH = None # Initialize to store the solution series
    current_iteration_solution = None  # Initialize to store the current iteration solution

    while delta_ws > tolerance_w and iteration_count < max_iterations_sat:
        iteration_count += 1

        # dehumidification acording saturation curve
        A_DH.loc['DH', 'θ2'] = psy.wsp(theta_s_guess)
        A_DH.loc['DH', 'w2'] = -1
        b_DH.loc['DH'] = psy.wsp(theta_s_guess) * theta_s_guess - psy.w(theta_s_guess, 1)

        # Convert to numpy arrays for solving
        A_np = A_DH.to_numpy()
        b_np = b_DH.to_numpy()

        # Solve the linear system
        # Use try-except to handle potential singular matrix errors
        try:
            x_DH = np.linalg.solve(A_np, b_np)
        except np.linalg.LinAlgError:
            print(f"  Error: Singular matrix in saturation iteration {iteration_count}.")
            current_iteration_solution = None # Store the current iteration solution as None
            break
        
        # Convert the solution array x_DH to a Pandas Series
        current_iteration_solution = pd.Series(x_DH, index=columns_DH)

        θ2_calc = current_iteration_solution['θ2'] # Calculate the temperature after cooling coil
        w_2_calc = current_iteration_solution['w2'] # Calculate the humidity ratio after cooling coil
    
        delta_ws = abs(psy.w(θ2_calc, 1)- w_2_calc) # Calculate the difference in humidity ratio

        # DEBUGGING PRINTS (English):
        print(f"Iteration: {iteration_count}")
        print(f"  Previous theta_s_guess (previous θ2_calc): {theta_s_guess:.5f}")
        print(f"  Current θ2_calc: {θ2_calc:.5f}")
        print(f"  Current w_2_calc: {w_2_calc:.7f}")
        print(f"  psy.w(θ2_calc, 1) (Saturation humidity at θ2_calc): {psy.w(θ2_calc, 1):.7f}")
        print(f"  delta_ws: {delta_ws:.7f}")
        print(f"  tolerance_w: {tolerance_w:.7f}")
        print(f"  Condition delta_ws > tolerance_w: {delta_ws > tolerance_w}")
        # END DEBUGGING PRINTS

        theta_s_guess = θ2_calc # Update for next iteration


    if current_iteration_solution is not None: # Check if the last iteration was successful
        if delta_ws <= tolerance_w: # Convergence achieved
            converged_sat = True
            solution_series_DH = current_iteration_solution 
            print(f"\nSaturation point converged after {iteration_count} iterations. Final ADP ≈ {theta_s_guess:.3f}°C")
        elif iteration_count >= max_iterations_sat: # Max iterations reached
            print(f"\nSaturation point did NOT converge after {max_iterations_sat} iterations. Last delta_ws = {delta_ws*1000:.5f} g/kg")
            solution_series_DH = None
    else: # If no solution was found in the last iteration
        print("\nNo successful iteration completed for saturation point (e.g., error on first try).")
        solution_series_DH = None

    # Convert the solution array x_DH to a Pandas Series
    if solution_series_DH is not None:
        print("\n--- Dehumidification Results ---")
        explanations = {
            'θ1': 'Temperature after mixing (MX1) [°C]',
            'w1': 'Humidity ratio after mixing (MX1) [kg/kg]',
            'θ2': 'Temperature after cooling coil (CC) [°C]',
            'w2': 'Humidity ratio after cooling coil (CC) [kg/kg]',
            'θ3': 'Temperature after mixing (MX2) [°C]',
            'w3': 'Humidity ratio after mixing (MX2) [kg/kg]',
            'θ4': 'Temperature after heating coil (HC) [°C]',
            'w4': 'Humidity ratio after heating coil (HC) [kg/kg]',
            'θ5': 'Temperature after vapor humidifier (VH) [°C]',
            'w5': 'Humidity ratio after vapor humidifier (VH) [kg/kg]',
            'θ6': 'Room temperature (Thermal Zone) [°C]',
            'w6': 'Room humidity ratio (Thermal Zone) [kg/kg]',
            'Qs_CC': 'Sensible heat load of cooling coil (CC) [W]',
            'Ql_CC': 'Latent heat load of cooling coil (CC) [W]',
            'Qt_CC': 'Total heat load of cooling coil (CC) [W]',
            'Qs_HC': 'Sensible heat load of heating coil (HC) [W]',
            'Qs_B': 'Sensible heat load of the building [W]',
            'Ql_B': 'Latent heat load of the building [W]'
        }
        for variable, value in solution_series_DH.items():
            explanation = explanations.get(variable, 'Unknown variable')
            print(f"{variable}: {value:.3f} ({explanation})")
    else:
        print("\nNo valid solution to display for Dehumidification.")

elif w_0 < w_6_sp:
    print('Humidification')
    print('======================')

    # Convert back to numpy arrays for solving
    A_np = A_H.to_numpy()
    b_np = b_H.to_numpy()

    # Solution
    x_H = np.linalg.solve(A_np, b_np)

    # Convert the solution array x_H to a Pandas Series
    solution_series_H = pd.Series(x_H, index=columns_H)
    

    # Dictionary to map variables to their explanations and units
    explanations = {
        'θ1': 'Temperature after mixing (MX1) [°C]',
        'w1': 'Humidity ratio after mixing (MX1) [kg/kg]',
        'θ2': 'Temperature after cooling coil (CC) [°C]',
        'w2': 'Humidity ratio after cooling coil (CC) [kg/kg]',
        'θ3': 'Temperature after mixing (MX2) [°C]',
        'w3': 'Humidity ratio after mixing (MX2) [kg/kg]',
        'θ4': 'Temperature after heating coil (HC) [°C]',
        'w4': 'Humidity ratio after heating coil (HC) [kg/kg]',
        'θ5': 'Temperature after vapor humidifier (VH) [°C]',
        'w5': 'Humidity ratio after vapor humidifier (VH) [kg/kg]',
        'θ6': 'Room temperature (Thermal Zone) [°C]',
        'w6': 'Room humidity ratio (Thermal Zone) [kg/kg]',
        'Qs_HC': 'Sensible heat load of heating coil (HC) [W]',
        'Ql_VC': 'Latent heat load of vapor humidifier (VH) [W]',
        'Qs_B': 'Sensible heat load of the building [W]',
        'Ql_B': 'Latent heat load of the building [W]'
    }

    # Print each entry with explanation and unit
    for variable, value in solution_series_H.items():
        explanation = explanations.get(variable, 'Unknown variable')
        print(f"{variable}: {value:.3f} ({explanation})")

else: 
    print('No humidification or dehumidification needed')

Dehumidification
Iteration: 1
  Previous theta_s_guess (previous θ2_calc): 10.00000
  Current θ2_calc: 15.37760
  Current w_2_calc: 0.0104130
  psy.w(θ2_calc, 1) (Saturation humidity at θ2_calc): 0.0109138
  delta_ws: 0.0005008
  tolerance_w: 0.0000000
  Condition delta_ws > tolerance_w: True
Iteration: 2
  Previous theta_s_guess (previous θ2_calc): 15.37760
  Current θ2_calc: 14.67494
  Current w_2_calc: 0.0104130
  psy.w(θ2_calc, 1) (Saturation humidity at θ2_calc): 0.0104229
  delta_ws: 0.0000099
  tolerance_w: 0.0000000
  Condition delta_ws > tolerance_w: True
Iteration: 3
  Previous theta_s_guess (previous θ2_calc): 14.67494
  Current θ2_calc: 14.66044
  Current w_2_calc: 0.0104130
  psy.w(θ2_calc, 1) (Saturation humidity at θ2_calc): 0.0104130
  delta_ws: 0.0000000
  tolerance_w: 0.0000000
  Condition delta_ws > tolerance_w: True
Iteration: 4
  Previous theta_s_guess (previous θ2_calc): 14.66044
  Current θ2_calc: 14.66044
  Current w_2_calc: 0.0104130
  psy.w(θ2_calc, 1) (Satura